# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [1]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [0]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [0]:
# --- Select Tensorflow 2.0 (only in Google Colab)
% tensorflow_version 2.x

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [4]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 276kB 7.1MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=b370866dda7acd05b4194ef671eaa500210258ce7a9cf2bb5d65a199ef983241
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [0]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers, callbacks
from jarvis.train import datasets, custom
from jarvis.utils.display import imshow

# Data

As in the tutorial, data for this assignment will consist of prostate MRI exams. In prior work, an algorithm was created to separate out different MRI sequences. In this current assignment, only T2-weighted images (isolated using the prior algorithm) will be used for segmentation. In prostate imaging, the T2-weighted sequence captures the greatest amount of anatomic detail and is thus ideal for delineation of prostate gland structures.

The following lines of code will download the dataset (if not already present). Since each algorithm below requires a different dataset, the required generators and inputs will be defined dyanically in the code blocks later in this notebook.

In [6]:
# --- Download dataset
datasets.download(name='mr/prostatex-seg')

[ 2020-05-12 18:24:19 ] [====================] 100.000% : Extracting archive (0001380 / 0001380) 

{'code': '/data/raw/mr_prostatex_seg', 'data': '/data/raw/mr_prostatex_seg'}

# Training

A total of three different network architectures will be tested. The goal is to compare the incremental benefit of several design choices. After building and training each model to convergence, do not forget to save each model as a separate `*.hdf5` file.

## 1. 2D U-Net (single step)

In this algorithm a standard 2D U-Net architecture will be used to perform prostate segmentation. This network is **identical** to the week 5 assignment. The algorithm input will include an entire full field-of-view `256 x 256` resolution 2D slice from a T2 weighted MR image. Key customizations to the standard U-Net architecture that should be implemented (as in the week 5 assignment) include:

* same padding (vs. valid padding)
* strided convolutions (vs. max-pooling)

### Create generators and inputs

In [0]:
# --- Original 256 x 256 (one-step)
configs = {'batch': {'size': 12}}
gen_train, gen_valid, client = datasets.prepare(name='mr/prostatex-seg', keyword='seg-256',configs = {'batch': {'size': 12}})
inputs = client.get_inputs(Input)

In [0]:
xs, ys= next(gen_train)
imshow(xs['dat'], ys['zones'], figsize=(12, 12))
print(ys['zones'].shape)
print(inputs['dat'].shape)

### Define model

In [0]:
kwargs_1 = {
    'kernel_size': (1,3,3),
    'padding': 'same'
}

convolution = lambda x, filters, strides: layers.Conv3D(filters = filters, strides = strides, **kwargs_1)(x)
norm = lambda x: layers.BatchNormalization()(x)
relu = lambda x: layers.LeakyReLU()(x)

conv1 = lambda filters, x: relu(norm(convolution(x, filters, strides = (1,1,1))))
conv2 = lambda filters, x: relu(norm(convolution(x, filters, strides = (1,2,2))))


concat = lambda a,b: layers.Concatenate()([a,b])

tran = lambda x, filters, strides: layers.Conv3DTranspose(filters = filters, strides = strides, **kwargs_1)(x)

tran2 = lambda filters, x: relu(norm(tran(x, filters, strides=(1, 2, 2))))


In [9]:
# --- Define model
l1 = conv1(8, inputs['dat'])  #the first layer doesnt go down. It gets the 8 nodes
l2 = conv1(16, conv2(16,l1))
l3 = conv1(32, conv2(32,l2))
l4 = conv1(64, conv2(64,l3))
l5 = conv1(128, conv2(128,l4))
l6 = tran2(64, l5)
l7 = tran2(32, conv1(64, concat(l6, l4)))
l8 = tran2(16, conv1(32, concat(l7, l3)))
l9 = tran2(8, conv1(16, concat(l8, l2)))
l10 = conv1(8, l9 + inputs['dat'] )

# --- Create logits
logits = {}
logits['zones'] = layers.Conv3D(filters=3, kernel_size = (1,3,3), name='zones', padding = 'same')(l10)

# --- Create model
model = Model(inputs=inputs, outputs=logits)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dat (InputLayer)                [(None, None, 256, 2 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, None, 256, 25 80          dat[0][0]                        
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, 256, 25 32          conv3d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, None, 256, 25 0           batch_normalization[0][0]        
______________________________________________________________________________________________

### Compile and train model

In [10]:
# --- Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss = {'zones': losses.SparseCategoricalCrossentropy(from_logits = True)},
    metrics = {'zones': custom.dsc(cls=2)},
     experimental_run_tf_function=False
)

client.load_data_in_memory()

callback = callbacks.EarlyStopping(monitor='val_dsc_2', patience = 8, restore_best_weights=True, mode= 'max')
# --- Train the model
model.fit(
    x = gen_train,
    steps_per_epoch = 100,
    epochs = 100,
    validation_data = gen_valid,
    validation_steps = 500,
    validation_freq = 4,
#    use_muiltiprocessing = True,
    callbacks = callback,
)

[ 2020-05-12 18:24:40 ] [====================] 100.000% : Iterating | 000342    Epoch 1/100
100/100 [==============================] - 25s 255ms/step - loss: 0.9872 - dsc_1: 0.0585 - dsc_2: 0.0055
Epoch 2/100
100/100 [==============================] - 26s 256ms/step - loss: 0.6048 - dsc_1: 0.3041 - dsc_2: 6.0399e-04
Epoch 3/100
100/100 [==============================] - 26s 258ms/step - loss: 0.3687 - dsc_1: 0.5669 - dsc_2: 0.0390
Epoch 4/100
100/100 [==============================] - 62s 623ms/step - loss: 0.2276 - dsc_1: 0.6752 - dsc_2: 0.2354 - val_loss: 0.2305 - val_dsc_1: 0.6497 - val_dsc_2: 0.2825
Epoch 5/100
100/100 [==============================] - 26s 258ms/step - loss: 0.1476 - dsc_1: 0.7590 - dsc_2: 0.4768
Epoch 6/100
100/100 [==============================] - 26s 257ms/step - loss: 0.1052 - dsc_1: 0.7862 - dsc_2: 0.5272
Epoch 7/100
100/100 [==============================] - 26s 257ms/step - loss: 0.0781 - dsc_1: 0.8087 - dsc_2: 0.5573
Epoch 8/100
100/100 [=================

## 2. 2D U-Net (multiple step)

In this algorithm, the output of the first 2D U-Net will be used to generated a cropped `128 x 128` resolution 2D slice centered around the prostate gland. This method effectively focuses the algorithm field-of-view to the area of interest and helps improve on inherent class imbalance associated with this task.

### Create generators and inputs

In [0]:
# --- Cropped 128 x 128 (multiple step)
gen_train, gen_valid, client = datasets.prepare(name='mr/prostatex-seg', keyword='seg-crp', configs = {'batch': {'size': 12}})
inputs = client.get_inputs(Input)

In [0]:
xs, ys= next(gen_train)
imshow(xs['dat'], ys['zones'], figsize=(12, 12))
print(ys['zones'].shape)
print(inputs['dat'].shape)

### Define model

In [12]:
# --- Define model

#l1_2step = conv1(4 , inputs['dat']) #the first layer doesnt go down. It gets the 8 nodes
l2_2step = conv1(4, inputs['dat'])
l3_2step = conv1(8, conv2(4,l2_2step))
l4_2step = conv1(16, conv2(8,l3_2step))
l5_2step = conv1(32, conv2(16,l4_2step))
l6_2step = conv1(64, conv2(32,l5_2step))


l7_2step = tran2(32, l6_2step)
l8_2step = tran2(16, conv1(32, concat(l7_2step, l5_2step)))
l9_2step = tran2(8, conv1(16, concat(l8_2step, l4_2step)))

l10_2step = tran2(4, conv1(8, concat(l9_2step, l3_2step)))
#l11_2step = tran2(4, conv1(32, concat(l10_2step, l2_2step)))
l11_2step = conv1(4, concat(l10_2step, inputs['dat']))
# --- Create logits

logits_2step = {}
logits_2step['zones'] = layers.Conv3D(filters=3, kernel_size = (1,3,3), name='zones', padding = 'same')(l11_2step)

# --- Create model
model2 = Model(inputs=inputs, outputs=logits_2step)
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dat (InputLayer)                [(None, None, 128, 1 0                                            
__________________________________________________________________________________________________
conv3d_13 (Conv3D)              (None, None, 128, 12 40          dat[0][0]                        
__________________________________________________________________________________________________
batch_normalization_17 (BatchNo (None, None, 128, 12 16          conv3d_13[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_17 (LeakyReLU)      (None, None, 128, 12 0           batch_normalization_17[0][0]     
____________________________________________________________________________________________

### Compile and train model

In [13]:
# --- Compile model
model2.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss = {'zones': losses.SparseCategoricalCrossentropy(from_logits = True)},
    metrics = {'zones': custom.dsc(cls=2)},
     experimental_run_tf_function=False
)

client.load_data_in_memory()

callback2 = callbacks.EarlyStopping(monitor='val_dsc_2', patience = 12, restore_best_weights=True, mode= 'max')
# --- Train the model
model2.fit(
    x = gen_train,
    steps_per_epoch = 150,
    epochs = 200,
    validation_data = gen_valid,
    validation_steps = 500,
    validation_freq = 4,
    #use_muiltiprocessing = True,
    callbacks = callback2,
)

[ 2020-05-12 19:18:35 ] [====================] 100.000% : Iterating | 000342    Epoch 1/200
150/150 [==============================] - 9s 60ms/step - loss: 0.9008 - dsc_1: 0.4840 - dsc_2: 0.0758
Epoch 2/200
150/150 [==============================] - 9s 59ms/step - loss: 0.5106 - dsc_1: 0.6873 - dsc_2: 0.1642
Epoch 3/200
150/150 [==============================] - 9s 59ms/step - loss: 0.3359 - dsc_1: 0.7714 - dsc_2: 0.4286
Epoch 4/200
150/150 [==============================] - 20s 135ms/step - loss: 0.2624 - dsc_1: 0.7930 - dsc_2: 0.5304 - val_loss: 0.2317 - val_dsc_1: 0.8003 - val_dsc_2: 0.5513
Epoch 5/200
150/150 [==============================] - 9s 60ms/step - loss: 0.2097 - dsc_1: 0.8295 - dsc_2: 0.5886
Epoch 6/200
150/150 [==============================] - 9s 58ms/step - loss: 0.1843 - dsc_1: 0.8338 - dsc_2: 0.6180
Epoch 7/200
150/150 [==============================] - 9s 59ms/step - loss: 0.1674 - dsc_1: 0.8377 - dsc_2: 0.6325
Epoch 8/200
150/150 [==============================] -

## 3. Custom architecture (multiple step)

Finally, using all customizations described in class, find a top-performing model that yields some incremental benefit over the two baseline models above. A multi-step approach (using the cropped `128 x 128` inputs) will tend to yield improved results. Additional modifications that be used include (but are not limited to):

* hybrid 3D/2D network
* residual connections
* added convolutions between contracting and expanding layers 

In [0]:
# --- Select shape
configs = {'specs': {'xs': {'dat': {'shape': [3, 128, 128, 1]}}}, 'batch': {'size': 12}}

# --- Cropped 128 x 128 (multiple step)
gen_train, gen_valid, client = datasets.prepare(name='mr/prostatex-seg', keyword='seg-crp', configs=configs)
inputs = client.get_inputs(Input)

### Define model

In [0]:
xs, ys= next(gen_train)
imshow(xs['dat'][0], figsize=(15, 15))
imshow(ys['zones'], figsize=(15, 15))
print(ys['zones'].shape)
print(xs['dat'].shape)

In [0]:
# --- Define kwargs dictionary
kwargs_2 = {
    'kernel_size': (1, 3, 3)}
# --- Define lambda functions
conv_3 = lambda x, filters, padding, strides : layers.Conv3D(filters=filters, strides=strides, padding = padding, **kwargs_2)(x)
norm_3 = lambda x : layers.BatchNormalization()(x)
relu_3 = lambda x : layers.LeakyReLU()(x)

conv_3z = lambda x, filters, padding, strides, kernel_size : layers.Conv3D(filters=filters, strides=strides, padding = padding, kernel_size = kernel_size)(x)

validconv = lambda x, filters, kernel_size : layers.Conv3D(filters=filters, kernel_size = kernel_size, padding = 'valid')(x)

# --- Define stride-1, stride-2 blocks
conv1_3 = lambda filters, x: relu_3(norm_3(conv_3(x, filters, padding = 'same', strides=1)))
conv2_3 = lambda filters, x: relu_3(norm_3(conv_3(x, filters, padding = 'same', strides=(1, 2, 2))))
conv3_3 = lambda filters, x, kernel_size : relu_3(norm_3(conv_3z(x, filters, kernel_size = kernel_size, padding = 'valid', strides=1)))
conv4_3 = lambda filters, x, kernel_size : relu_3(norm_3(conv_3z(x, filters, kernel_size = kernel_size, padding = 'valid', strides=1)))


validconvl = lambda filters, x, kernel_size : relu_3(norm_3(validconv(x, filters, kernel_size)))

# --- Define single transpose
tran_3 = lambda x, filters, strides, padding : layers.Conv3DTranspose(filters=filters, strides=strides, padding = padding, **kwargs_2)(x)

# --- Define transpose block
tran2_3 = lambda filters, x : relu_3(norm_3(tran_3(x, filters, strides=(1, 2, 2), padding = 'same')))
tran3_3 = lambda filters, x : relu_3(norm_3(tran_3(x, filters, strides=(1, 4, 4), padding = 'same')))

concat = lambda a,b: layers.Concatenate()([a,b])

In [25]:
# --- Define model

a = 10

l2_3step = conv1_3(8*a, inputs['dat'])
l3_3step = conv1_3(16*a, conv2_3(8*a,l2_3step))
l4_3step = conv1_3(32*a, conv2_3(16*a,l3_3step))
l5_3step = conv3_3(64*a, conv2_3(32*a,l4_3step), (2,1,1))
l6_3step = conv3_3(128*a, conv2_3(64*a,l5_3step), (2,1,1))


l7_3step = tran2_3(64*a, l6_3step)
l8_3step = tran2_3(32*a, conv1_3(64*a, conv1_3(64*a, l7_3step + conv3_3(64*a, l5_3step, (2,1,1))))) #residual not concat
l9_3step = tran2_3(16*a, conv1_3(32*a, conv1_3(32*a, l8_3step + conv3_3(32*a, l4_3step, (3,1,1))))) #residual not concat
 
l10_3step = tran2_3(8*a, conv1_3(16*a, conv1_3(16*a,l9_3step + conv3_3(16*a, l3_3step, (3,1,1)))))



# --- Create logits
logits_3step = {}
logits_3step['zones'] = layers.Conv3D(filters=3, kernel_size = (1,3,3), name='zones', padding = 'same')(l10_3step)


# --- Create model
model3 = Model(inputs=inputs, outputs=logits_3step)
model3.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dat (InputLayer)                [(None, None, 128, 1 0                                            
__________________________________________________________________________________________________
conv3d_62 (Conv3D)              (None, None, 128, 12 800         dat[0][0]                        
__________________________________________________________________________________________________
batch_normalization_78 (BatchNo (None, None, 128, 12 320         conv3d_62[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_78 (LeakyReLU)      (None, None, 128, 12 0           batch_normalization_78[0][0]     
____________________________________________________________________________________________

### Compile and train model

In [26]:
model3.compile(
    optimizer=optimizers.Adam(learning_rate=2e-5),
    loss = {'zones': losses.SparseCategoricalCrossentropy(from_logits = True)},
    metrics = {'zones': custom.dsc(cls=2)},
     experimental_run_tf_function=False
)

client.load_data_in_memory()

callback3 = callbacks.EarlyStopping(monitor='val_dsc_2', patience = 10, restore_best_weights=True, mode= 'max')
# --- Train the model
model3.fit(
    x = gen_train,
    steps_per_epoch = 100,
    epochs = 200,
    validation_data = gen_valid,
    validation_steps = 500,
    validation_freq = 4,
#    use_muiltiprocessing = True,
    callbacks = callback3
)

[ 2020-05-12 20:41:59 ] [====================] 100.000% : Iterating | 000342    Epoch 1/200
100/100 [==============================] - 126s 1s/step - loss: 0.7924 - dsc_1: 0.5377 - dsc_2: 0.2051
Epoch 2/200
100/100 [==============================] - 126s 1s/step - loss: 0.3353 - dsc_1: 0.7870 - dsc_2: 0.4710
Epoch 3/200
100/100 [==============================] - 126s 1s/step - loss: 0.2478 - dsc_1: 0.8244 - dsc_2: 0.5847
Epoch 4/200
100/100 [==============================] - 330s 3s/step - loss: 0.1977 - dsc_1: 0.8507 - dsc_2: 0.6310 - val_loss: 0.2536 - val_dsc_1: 0.7441 - val_dsc_2: 0.4472
Epoch 5/200
100/100 [==============================] - 126s 1s/step - loss: 0.1678 - dsc_1: 0.8666 - dsc_2: 0.6573
Epoch 6/200
100/100 [==============================] - 127s 1s/step - loss: 0.1510 - dsc_1: 0.8728 - dsc_2: 0.6815
Epoch 7/200
100/100 [==============================] - 126s 1s/step - loss: 0.1345 - dsc_1: 0.8825 - dsc_2: 0.6974
Epoch 8/200
100/100 [==============================] - 3

# Evaluation

For each of the three models, the following metrics should be calculated for **both the training and validation** cohorts:

* Dice score, mean
* Dice score, median
* Dice score, 25th percentile
* Dice score, 75th percentile

The Dice score values should be calculated both for peripheral and transitional zone (class 1 and 2); the Dice score for background does not need to be evaluated. As in prior assignments, accuracy is determined on a patient by patient (volume by volume) basis, so please calculate the Dice score values on the entire 3D volume (not slice-by-slice).

### Performance

The following minimum performance metrics must be met for full credit:

1. 2D U-Net, single step (full 256 x 256)

* peripheral zone: mean Dice score > 0.75
* transitional zone: mean Dice score > 0.55

2. 2D U-Net, multiple step (cropped)

* peripheral zone: mean Dice score > 0.80
* transitional zone: mean Dice score > 0.60

3. Custom architecture

* peripeheral zone: mean Dice score > 0.85
* transitional zone: mean Dice score > 0.65

In [0]:
def dice(y_true, y_pred, c=1, epsilon=1):
    """
    Method to calculate the Dice score coefficient for given class
    
    :params
    
      (np.ndarray) y_true : ground-truth label
      (np.ndarray) y_pred : predicted logits scores
      (int)             c : class to calculate DSC on
    
    """
    assert y_true.ndim == y_pred.ndim
    
    true = y_true[..., 0] == c
    pred = np.argmax(y_pred, axis=-1) == c 

    A = np.count_nonzero(true & pred) * 2
    B = np.count_nonzero(true) + np.count_nonzero(pred) + epsilon
    
    return A / B

In [28]:
dsc_pz_1 = []
dsc_tz_1 = []

dsc_pz_2 = []
dsc_tz_2 = []

dsc_pz_3 = []
dsc_tz_3 = []

gen_train, gen_valid, client = datasets.prepare(name='mr/prostatex-seg', keyword='seg-256')
inputs = client.get_inputs(Input)
test_train, test_valid = client.create_generators(test=True, expand=True)

for x, y in test_valid:
    
#    print(x['dat'].shape)
#    print(y['zones'].shape)
#    imshow(xs['dat'][0], figsize=(12, 12))
#    imshow(ys['zones'], figsize=(12, 12))
    # --- Predict)
    logits_1 = model.predict(x['dat'])
    if type(logits_1) is dict:
        logits_1 = logits_1['zones'] 

    # --- Argmax
    dsc_pz_1.append(dice(y['zones'][0], logits_1[0], c=1))
    dsc_tz_1.append(dice(y['zones'][0], logits_1[0], c=2))

# --- Select shape
configs = {'specs': {'xs': {'dat': {'shape': [3, 128, 128, 1]}}}, 'batch': {'size': 12}}

# --- Cropped 128 x 128 (multiple step)
gen_train, gen_valid, client = datasets.prepare(name='mr/prostatex-seg', keyword='seg-crp', configs=configs)
inputs = client.get_inputs(Input)
test_train, test_valid = client.create_generators(test=True, expand=True)

for x, y in test_valid:
    
#    print(x['dat'].shape)
#    print(y['zones'].shape)
#    imshow(xs['dat'][0], figsize=(12, 12))
#    imshow(ys['zones'], figsize=(12, 12))
    # --- Predict
    x['dat_3'] = np.pad(x['dat'], ((0, 0), (1, 1), (0, 0), (0, 0), (0, 0)))
    logits_2 = model2.predict(x['dat'])
    if type(logits_1) is dict:
        logits_1 = logits_1['zones']
    if type(logits_2) is dict:
        logits_2 = logits_2['zones']     

    # --- Argmax
    dsc_pz_2.append(dice(y['zones'][0], logits_2[0], c=1))
    dsc_tz_2.append(dice(y['zones'][0], logits_2[0], c=2))


    logits_3 = model3.predict(x['dat_3'])

    if type(logits_3) is dict:
        logits_3 = logits_3['zones']    

    dsc_pz_3.append(dice(y['zones'][0], logits_3[0], c=1))
    dsc_tz_3.append(dice(y['zones'][0], logits_3[0], c=2))




dsc_pz_1 = np.array(dsc_pz_1)
dsc_tz_1 = np.array(dsc_tz_1)

dsc_pz_2 = np.array(dsc_pz_2)
dsc_tz_2 = np.array(dsc_tz_2)

dsc_pz_3 = np.array(dsc_pz_3)
dsc_tz_3 = np.array(dsc_tz_3)


df = pd.DataFrame(index=np.arange(dsc_tz_3.size))


df['dsc_pz_1'] = dsc_pz_1
df['dsc_tz_1'] = dsc_tz_1

df['dsc_pz_2'] = dsc_pz_2
df['dsc_tz_2'] = dsc_tz_2

df['dsc_pz_3'] = dsc_pz_3
df['dsc_tz_3'] = dsc_tz_3

df['dsc_pz_1.mean'] = df['dsc_pz_1'].mean()
df['dsc_tz_1.mean'] = df['dsc_tz_1'].mean()

df['dsc_pz_2.mean'] = df['dsc_pz_2'].mean()
df['dsc_tz_2.mean'] = df['dsc_tz_2'].mean()

df['dsc_pz_3.mean'] = df['dsc_pz_3'].mean()
df['dsc_tz_3.mean'] = df['dsc_tz_3'].mean()


df['dsc_pz_1.median'] = df['dsc_pz_1'].median()
df['dsc_tz_1.median'] = df['dsc_tz_1'].median()

df['dsc_pz_2.median'] = df['dsc_pz_2'].median()
df['dsc_tz_2.median'] = df['dsc_tz_2'].median()

df['dsc_pz_3.median'] = df['dsc_pz_3'].median()
df['dsc_tz_3.median'] = df['dsc_tz_3'].median()

df['dsc_pz_1.quantile_25'] = df['dsc_pz_1'].quantile(0.25)
df['dsc_tz_1.quantile_25'] = df['dsc_tz_1'].quantile(0.25)

df['dsc_pz_2.quantile_25'] = df['dsc_pz_2'].quantile(0.25)
df['dsc_tz_2.quantile_25'] = df['dsc_tz_2'].quantile(0.25)

df['dsc_pz_3.quantile_25'] = df['dsc_pz_3'].quantile(0.25)
df['dsc_tz_3.quantile_25'] = df['dsc_tz_3'].quantile(0.25)

df['dsc_pz_1.quantile_75'] = df['dsc_pz_1'].quantile(0.75)
df['dsc_tz_1.quantile_75'] = df['dsc_tz_1'].quantile(0.75)

df['dsc_pz_2.quantile_75'] = df['dsc_pz_2'].quantile(0.75)
df['dsc_tz_2.quantile_75'] = df['dsc_tz_2'].quantile(0.75)

df['dsc_pz_3.quantile_75'] = df['dsc_pz_3'].quantile(0.75)
df['dsc_tz_3.quantile_75'] = df['dsc_tz_3'].quantile(0.75)



# --- Print accuracy

print('\n\n')

print(df['dsc_pz_1'].mean())
print(df['dsc_tz_1'].mean())

print(df['dsc_pz_2'].mean())
print(df['dsc_tz_2'].mean())

print(df['dsc_pz_3'].mean())
print(df['dsc_tz_3'].mean())

print('\n\n')


print(df['dsc_pz_1'].median())
print(df['dsc_tz_1'].median())

print(df['dsc_pz_2'].median())
print(df['dsc_tz_2'].median())

print(df['dsc_pz_3'].median())
print(df['dsc_tz_3'].median())


print('\n\n')


print(df['dsc_pz_1'].quantile(0.25))
print(df['dsc_tz_1'].quantile(0.25))

print(df['dsc_pz_2'].quantile(0.25))
print(df['dsc_tz_2'].quantile(0.25))

print(df['dsc_pz_3'].quantile(0.25))
print(df['dsc_tz_3'].quantile(0.25))

print('\n\n')


print(df['dsc_pz_1'].quantile(0.75))
print(df['dsc_tz_1'].quantile(0.75))

print(df['dsc_pz_2'].quantile(0.75))
print(df['dsc_tz_2'].quantile(0.75))

print(df['dsc_pz_3'].quantile(0.75))
print(df['dsc_tz_3'].quantile(0.75))

[ 2020-05-13 01:05:23 ] [=>..................] 7.246% : Iterating | 000005      WARNING:tensorflow:5 out of the last 5 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7fb6a0ef6d08> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.
[ 2020-05-13 01:05:26 ] [=>..................] 8.696% : Iterating | 000006      WARNING:tensorflow:6 out of the last 6 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7fb6a0ef6d08> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python

### Results

When ready, create a `*.csv` file with your compiled **training and validation** cohort statistics for the three different models. Consider the following table format (although any format that contains the required information is sufficient):

```
          TRANSITIONAL ZONE                       PERIPHERAL ZONE
          mean | median | 25th-tile | 75th-tile | mean | median | 25th-tile | 75th-tile
model 1
model 2
model 3
```

As above, tables for both training and validation should be provided.

In [0]:
# --- Create *.csv
                              
# --- Serialize *.csv
fname = '{}/models/midterm/model.hdf5'.format(MOUNT_ROOT)
fname2 = '{}/models/midterm/model2.hdf5'.format(MOUNT_ROOT)
fname3 = '{}/models/midterm/model3.hdf5'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
df.to_csv('midterm.csv')
model.save(fname)
model2.save(fname2)
model3.save(fname3)